In [5]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch.nn import BatchNorm1d, Linear, ReLU, Sequential

from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GINConv, global_add_pool

path = osp.join('data', 'TU')
# dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES').shuffle()
dataset = TUDataset(path, name='MUTAG').shuffle()

train_dataset = dataset[len(dataset) // 10:]
test_dataset = dataset[:len(dataset) // 10]

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128)


class Net(torch.nn.Module):
    def __init__(self, in_channels, dim, out_channels):
        super().__init__()

        self.conv1 = GINConv(
            Sequential(Linear(in_channels, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU()))

        self.conv2 = GINConv(
            Sequential(Linear(dim, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU()))

        self.conv3 = GINConv(
            Sequential(Linear(dim, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU()))

        self.conv4 = GINConv(
            Sequential(Linear(dim, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU()))

        self.conv5 = GINConv(
            Sequential(Linear(dim, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU()))

        self.lin1 = Linear(dim, dim)
        self.lin2 = Linear(dim, out_channels)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.conv3(x, edge_index)
        x = self.conv4(x, edge_index)
        x = self.conv5(x, edge_index)
        x = global_add_pool(x, batch)
        x = self.lin1(x).relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dataset.num_features, 32, dataset.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x, data.edge_index, data.batch)
        loss = F.nll_loss(output, data.y)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(loader):
    model.eval()

    total_correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        total_correct += int((out.argmax(-1) == data.y).sum())
    return total_correct / len(loader.dataset)


for epoch in range(1, 101):
    loss = train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f} '
          f'Test Acc: {test_acc:.4f}')

Epoch: 001, Loss: 0.7186, Train Acc: 0.6588 Test Acc: 0.7222
Epoch: 002, Loss: 0.5876, Train Acc: 0.6588 Test Acc: 0.7222
Epoch: 003, Loss: 0.4562, Train Acc: 0.6588 Test Acc: 0.7222
Epoch: 004, Loss: 0.4385, Train Acc: 0.6588 Test Acc: 0.7222
Epoch: 005, Loss: 0.4156, Train Acc: 0.6588 Test Acc: 0.7222
Epoch: 006, Loss: 0.3732, Train Acc: 0.6588 Test Acc: 0.7222
Epoch: 007, Loss: 0.3066, Train Acc: 0.6588 Test Acc: 0.7222
Epoch: 008, Loss: 0.3186, Train Acc: 0.6588 Test Acc: 0.7222
Epoch: 009, Loss: 0.3074, Train Acc: 0.6588 Test Acc: 0.7222
Epoch: 010, Loss: 0.3207, Train Acc: 0.6706 Test Acc: 0.7222
Epoch: 011, Loss: 0.3224, Train Acc: 0.7059 Test Acc: 0.7778
Epoch: 012, Loss: 0.2930, Train Acc: 0.7235 Test Acc: 0.7778
Epoch: 013, Loss: 0.3104, Train Acc: 0.7412 Test Acc: 0.7778
Epoch: 014, Loss: 0.2723, Train Acc: 0.7588 Test Acc: 0.7778
Epoch: 015, Loss: 0.3096, Train Acc: 0.7471 Test Acc: 0.7778
Epoch: 016, Loss: 0.2920, Train Acc: 0.7529 Test Acc: 0.7778
Epoch: 017, Loss: 0.2932

In [6]:
from torch_geometric.nn import GraphConv
print(model)

Net(
  (conv1): GINConv(nn=Sequential(
    (0): Linear(in_features=7, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=32, out_features=32, bias=True)
    (4): ReLU()
  ))
  (conv2): GINConv(nn=Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=32, out_features=32, bias=True)
    (4): ReLU()
  ))
  (conv3): GINConv(nn=Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=32, out_features=32, bias=True)
    (4): ReLU()
  ))
  (conv4): GINConv(nn=Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True,